In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [18]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,BatchNormalization
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14680403688388806390
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 358744064
locality {
  bus_id: 1
}
incarnation: 2763980261787646926
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:05.0"
]


In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded



Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`. 

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .



### Vocabulary


In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"



## Preprocess
Tokenization

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk
tests.test_tokenize(tokenize)


text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding 

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)


test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline


In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344



### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation. 

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: Basic RNN 

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 111us/step - loss: 3.4108 - acc: 0.4193 - val_loss: nan - val_acc: 0.4566
Epoch 2/10
110288/110288 [==============================] - 10s 87us/step - loss: 2.4486 - acc: 0.4714 - val_loss: nan - val_acc: 0.4830
Epoch 3/10
110288/110288 [==============================] - 10s 88us/step - loss: 2.2019 - acc: 0.5088 - val_loss: nan - val_acc: 0.5431
Epoch 4/10
110288/110288 [==============================] - 10s 88us/step - loss: 1.9563 - acc: 0.5542 - val_loss: nan - val_acc: 0.5697
Epoch 5/10
110288/110288 [==============================] - 10s 87us/step - loss: 1.7824 - acc: 0.5757 - val_loss: nan - val_acc: 0.5831
Epoch 6/10
110288/110288 [==============================] - 10s 87us/step - loss: 1.6865 - acc: 0.5854 - val_loss: nan - val_acc: 0.5904
Epoch 7/10
110288/110288 [==============================] - 10s 87us/step - loss: 1.6241 - acc: 0.5908 - val_loss: nan - val_a

### Model 2: Embedding 

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

        
    embedding_size = 128
    rnn_units = 256
    learning_rate = 1e-3
    
    model = Sequential()
    
    model.add(Embedding(input_dim=english_vocab_size, output_dim=embedding_size, input_length=input_shape[1]))
    
    model.add(GRU(rnn_units, return_sequences=True))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model





tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 17s 150us/step - loss: 3.1671 - acc: 0.4295 - val_loss: nan - val_acc: 0.5010
Epoch 2/10
110288/110288 [==============================] - 16s 148us/step - loss: 1.8798 - acc: 0.5745 - val_loss: nan - val_acc: 0.6529
Epoch 3/10
110288/110288 [==============================] - 16s 148us/step - loss: 1.2003 - acc: 0.7083 - val_loss: nan - val_acc: 0.7612
Epoch 4/10
110288/110288 [==============================] - 16s 148us/step - loss: 0.8403 - acc: 0.7844 - val_loss: nan - val_acc: 0.8085
Epoch 5/10
110288/110288 [==============================] - 16s 148us/step - loss: 0.6319 - acc: 0.8241 - val_loss: nan - val_acc: 0.8366
Epoch 6/10
110288/110288 [==============================] - 16s 148us/step - loss: 0.5120 - acc: 0.8506 - val_loss: nan - val_acc: 0.8632
Epoch 7/10
110288/110288 [==============================] - 16s 149us/step - loss: 0.4338 - acc: 0.8707 - val_loss: nan -

### Model 3: Bidirectional RNN

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    rnn_units = 256
    learning_rate = 1e-3

    model = Sequential()

    model.add(Bidirectional(GRU(rnn_units, return_sequences=True), input_shape=input_shape[1:]))

    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model



tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


bd_rnn_model = bd_model(
    tmp_x.shape,  
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 24s 218us/step - loss: 2.1429 - acc: 0.5454 - val_loss: nan - val_acc: 0.6098
Epoch 2/10
110288/110288 [==============================] - 24s 215us/step - loss: 1.3593 - acc: 0.6274 - val_loss: nan - val_acc: 0.6430
Epoch 3/10
110288/110288 [==============================] - 24s 215us/step - loss: 1.2082 - acc: 0.6525 - val_loss: nan - val_acc: 0.6638
Epoch 4/10
110288/110288 [==============================] - 24s 215us/step - loss: 1.1141 - acc: 0.6738 - val_loss: nan - val_acc: 0.6809TA: 17s - loss: 1.1455 - acc: 0. - E - ETA: 8s - loss: 1.1291 - ac - ETA
Epoch 5/10
110288/110288 [==============================] - 24s 215us/step - loss: 1.0410 - acc: 0.6865 - val_loss: nan - val_acc: 0.6921
Epoch 6/10
110288/110288 [==============================] - 24s 215us/step - loss: 0.9821 - acc: 0.6957 - val_loss: nan - val_acc: 0.7000
Epoch 7/10
110288/110288 [=======================

### Model 4: Encoder-Decoder 

In [14]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    learning_rate = 1e-3
    
    encoder_inputs = Input(shape=input_shape[1:])
    encoder_gru = GRU(256, return_state=True)
    encoder_outputs, encoder_state = encoder_gru(encoder_inputs)

  
    decoder_state = RepeatVector(output_sequence_length)(encoder_state)

    
    decoder_gru = GRU(256, return_sequences=True)
    decoder_outputs = decoder_gru(decoder_state, initial_state=encoder_state)

    
    decoder_dense = Dense(french_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    
    model = Model(inputs=encoder_inputs, outputs=decoder_outputs)

    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

tmp_x = preproc_english_sentences
tmp_x = tmp_x.reshape(preproc_english_sentences.shape[0],preproc_english_sentences.shape[1],1)

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)



decoder_targets = preproc_french_sentences


encdec_rnn_model.fit(tmp_x, decoder_targets, batch_size=1024, epochs=10, validation_split=0.2)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))




Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 25s 231us/step - loss: 2.5359 - acc: 0.4873 - val_loss: nan - val_acc: 0.5541
Epoch 2/10
110288/110288 [==============================] - 25s 224us/step - loss: 1.6563 - acc: 0.5747 - val_loss: nan - val_acc: 0.5898
Epoch 3/10
110288/110288 [==============================] - 25s 224us/step - loss: 1.4479 - acc: 0.6014 - val_loss: nan - val_acc: 0.6190
Epoch 4/10
110288/110288 [==============================] - 25s 225us/step - loss: 1.3265 - acc: 0.6275 - val_loss: nan - val_acc: 0.6276
Epoch 5/10
110288/110288 [==============================] - 25s 225us/step - loss: 1.2549 - acc: 0.6381 - val_loss: nan - val_acc: 0.6452
Epoch 6/10
110288/110288 [==============================] - 25s 225us/step - loss: 1.2078 - acc: 0.6456 - val_loss: nan - val_acc: 0.6509
Epoch 7/10
110288/110288 [==============================] - 25s 225us/step - loss: 1.1782 - acc: 0.6530 - val_loss: nan -

### Model 5: Custom 

In [19]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 1e-3
    
    encoder_input = Input(shape=input_shape[1:])
    encoder_embed = Embedding(input_dim=english_vocab_size, output_dim=128)(encoder_input)
    
    
    encoder_embed = BatchNormalization()(encoder_embed)
    
    encoder_rnn = Bidirectional(GRU(256, return_sequences=False))(encoder_embed)
    
    
    encoder_rnn = BatchNormalization()(encoder_rnn)

    decoder_input = RepeatVector(output_sequence_length)(encoder_rnn)
    
    decoder_rnn = GRU(256, return_sequences=True)(decoder_input)
    decoder_output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder_rnn)

    
    model = Model(inputs=encoder_input, outputs=decoder_output)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model



tmp_x = preproc_english_sentences

model_final_rnn = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print('Final Model Loaded')

model_final_rnn.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


print(logits_to_text(model_final_rnn.predict(tmp_x[:1])[0], french_tokenizer))



Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 38s 344us/step - loss: 2.3658 - acc: 0.4768 - val_loss: nan - val_acc: 0.0932
Epoch 2/10
110288/110288 [==============================] - 37s 332us/step - loss: 1.5400 - acc: 0.5987 - val_loss: nan - val_acc: 0.4100
Epoch 3/10
110288/110288 [==============================] - 37s 333us/step - loss: 1.0972 - acc: 0.6885 - val_loss: nan - val_acc: 0.4155
Epoch 4/10
110288/110288 [==============================] - 37s 333us/step - loss: 0.7900 - acc: 0.7691 - val_loss: nan - val_acc: 0.4503
Epoch 5/10
110288/110288 [==============================] - 37s 333us/step - loss: 0.5822 - acc: 0.8281 - val_loss: nan - val_acc: 0.5286
Epoch 6/10
110288/110288 [==============================] - 37s 333us/step - loss: 0.4476 - acc: 0.8690 - val_loss: nan - val_acc: 0.7694
Epoch 7/10
110288/110288 [==============================] - 37s 334us/step - loss: 0.3590 - acc: 0.899

## Prediction

In [20]:
def final_predictions(x, y, x_tk, y_tk):
 
    model = model_final(
    x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
    
    model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 38s 346us/step - loss: 2.3613 - acc: 0.4775 - val_loss: nan - val_acc: 0.2767
Epoch 2/10
110288/110288 [==============================] - 37s 333us/step - loss: 1.5162 - acc: 0.6009 - val_loss: nan - val_acc: 0.4093
Epoch 3/10
110288/110288 [==============================] - 37s 333us/step - loss: 1.0625 - acc: 0.6957 - val_loss: nan - val_acc: 0.4160
Epoch 4/10
110288/110288 [==============================] - 37s 333us/step - loss: 0.7472 - acc: 0.7839 - val_loss: nan - val_acc: 0.4490
Epoch 5/10
110288/110288 [==============================] - 37s 332us/step - loss: 0.5728 - acc: 0.8301 - val_loss: nan - val_acc: 0.5862
Epoch 6/10
110288/110288 [==============================] - 37s 332us/step - loss: 0.4463 - acc: 0.8695 - val_loss: nan - val_acc: 0.7802
Epoch 7/10
110288/110288 [==============================] - 37s 334us/step - loss: 0.3569 - acc: 0.8984 - val_loss: nan -